In [136]:
from nltk import ngrams, word_tokenize
import pickle
import os

In [137]:
with open('hm3_files/English.unigram.pickle', 'rb') as handle:
    unigram_english_dict = pickle.load(handle)

In [138]:
with open('hm3_files/English.bigram.pickle', 'rb') as handle:
    bigram_english_dict = pickle.load(handle)

In [139]:
with open('hm3_files/French.unigram.pickle', 'rb') as handle:
    unigram_french_dict = pickle.load(handle)

In [140]:
with open('hm3_files/French.bigram.pickle', 'rb') as handle:
    bigram_french_dict = pickle.load(handle)

In [141]:
with open('hm3_files/Italian.unigram.pickle', 'rb') as handle:
    unigram_italian_dict = pickle.load(handle)

In [142]:
with open('hm3_files/Italian.bigram.pickle', 'rb') as handle:
    bigram_italian_dict = pickle.load(handle)

In [143]:
vocabulary_size = (float)(len(unigram_english_dict) + len(unigram_french_dict) + len(unigram_italian_dict))
vocabulary_size = float(vocabulary_size)

In [144]:
def get_bigram_probability(bigram_phrase, first_word, bigram_dict, unigram_dict):
    bigram_count = bigram_dict.get(bigram_phrase)
    if bigram_count is None:
        bigram_count = 0
    bigram_count = float(bigram_count)
    first_word_count = unigram_dict.get(first_word)
    if first_word_count is None:
        first_word_count = 0
    first_word_count = float(first_word_count)
    return (bigram_count + 1) / (first_word_count + vocabulary_size)

In [145]:
def get_language_probability(bigram_list, first_words, bigram_dict, unigram_dict):
    result = 1.0
    index = 0
    for bigram in bigram_list:
        result *= get_bigram_probability(bigram, first_words[index], bigram_dict, unigram_dict)
        index += 1
    return result

In [146]:
solution_dict = dict()
with open('hm3_files/LangId.sol') as f:
    for line in f:
       (key, val) = line.split()
       solution_dict[int(key)] = val
        
line_no = 1
result_dict = dict()
correct = 0
incorrect_line_numbers = []

with open('hm3_files/LangId.test', encoding='utf8') as f:
    for line in f:
        tokens = word_tokenize(line.lower())
        bigrams = ngrams(tokens, 2)  
        bigram_list = []   
        first_words = []
        for b in bigrams:
            bigram_list.append(' '.join(b))
            first_words.append(b[0])
            
        english_prob = get_language_probability(bigram_list, first_words, bigram_english_dict, unigram_english_dict)
        french_prob = get_language_probability(bigram_list, first_words, bigram_french_dict, unigram_french_dict)
        italian_prob = get_language_probability(bigram_list, first_words, bigram_italian_dict, unigram_italian_dict)
        
        max_prob = max(english_prob, french_prob, italian_prob)
        if max_prob == english_prob:
            result_dict[line_no] = 'English'
        elif max_prob == french_prob:
            result_dict[line_no] = 'French'
        else:
            result_dict[line_no] = 'Italian'
        
        if solution_dict[line_no] == result_dict[line_no]:
            correct += 1
        else:
            incorrect_line_numbers.append(line_no)
            
        line_no += 1    

In [147]:
with open('hm3_files/LangId.result', 'w') as f:
    for (key, val) in result_dict.items():
        f.write(' '.join([str(key), val]) + os.linesep)   

In [148]:
print('Accuracy: {:2.2f}%'.format(float(correct) * 100 / len(solution_dict)))

Accuracy: 98.00%


In [150]:
print('Line numbers for incorrectly classified languages: {}'.format(str(incorrect_line_numbers)))

Line numbers for incorrectly classified languages: [24, 187, 191, 247, 277, 279]
